In [1]:
#Run this if you get an error of no yelpapi existing
#!pip install yelpapi
#Run this if you don't have tqdm installed
#!pip install tqdm

In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [3]:
with open('/Users/joseph/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [4]:
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [5]:
help(yelp_api.search_query)

Help on method search_query in module yelpapi.yelpapi:

search_query(**kwargs) method of yelpapi.yelpapi.YelpAPI instance
    Query the Yelp Search API.
    
    documentation: https://www.yelp.com/developers/documentation/v3/business_search
    
    required parameters:
        * one of either:
            * location - text specifying a location to search for
            * latitude and longitude



In [6]:
# use our yelp_api variable's search_query method to perform our API call
LOCATION = "Little Rock, AR"
TERM = "burger"
search_results = yelp_api.search_query(location = LOCATION,
                                       term = TERM)
print(type(search_results))
search_results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [7]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = f"Data/results_in_progress_{LOCATION}_{TERM}.json"
JSON_FILE

'Data/results_in_progress_Little Rock, AR_burger.json'

In [8]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")


[i] Data/results_in_progress_Little Rock, AR_burger.json not found. Saving empty list to file.


In [9]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [10]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [11]:
## How many results total?
total_results = results['total']
total_results

420

In [12]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [13]:
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

21

In [14]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [15]:
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/21 [00:00<?, ?it/s]

In [16]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)

  0%|          | 0/21 [00:00<?, ?it/s]

In [17]:
def create_json_file(JSON_FILE, delete_if_exists = False):
    ##Checking if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    ##If exists:
    if file_exists == True:
        if delete_if_exists == True:
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            #delete file
            os.remove(JSON_FILE)
            #Recursive call to function after old file is deleted
            create_json_file(JSON_FILE, delete_if_exists = False)
        else:
            print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
    ## If it does not exist
    else:
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        #Creating any needed folders
        #Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        # If JSON_FILE included a folder:
        if len(folder) > 0:
            #Make a folder
            os.makedirs(folder, exist_ok = True)
        #Save empty list to start the json file
        with open(JSON_FILE, "w") as f:
            json.dump([],f)

In [18]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_Little Rock, AR_burger.json already exists. Deleting previous file...
[i] Data/results_in_progress_Little Rock, AR_burger.json not found. Saving empty list to new file.
- 0 previous results found.


21

In [19]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)



  0%|          | 0/21 [00:00<?, ?it/s]

In [20]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,-ihewqJrusrWq9hj4jaIiA,big-orange-little-rock-2,Big Orange,https://s3-media2.fl.yelpcdn.com/bphoto/Ht_Yd_...,False,https://www.yelp.com/biz/big-orange-little-roc...,602,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"{'latitude': 34.7531331313556, 'longitude': -9...",[delivery],$$,"{'address1': '207 N University Ave', 'address2...",+15013798715,(501) 379-8715,3308.868005
1,hI0cnPhRvF5pVP3BEtecTg,davids-burgers-little-rock,David's Burgers,https://s3-media2.fl.yelpcdn.com/bphoto/eUn_Bg...,False,https://www.yelp.com/biz/davids-burgers-little...,195,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"{'latitude': 34.753454, 'longitude': -92.406499}",[delivery],$$,"{'address1': '101 S Bowman Rd', 'address2': ''...",+15012278333,(501) 227-8333,7038.847918
2,S3W6tw1i4jLLrmja_g1lzg,ye-pauls-soul-bodega-and-grill-little-rock-3,Ye Paul's Soul Bodega & Grill,https://s3-media1.fl.yelpcdn.com/bphoto/gYtX9K...,False,https://www.yelp.com/biz/ye-pauls-soul-bodega-...,38,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",5.0,"{'latitude': 34.70749987605255, 'longitude': -...","[pickup, delivery]",NaN,"{'address1': '4424 Frazier Pike', 'address2': ...",+15017471551,(501) 747-1551,10545.103583
3,vDmRK_D_GbKo58D2SJoWdg,north-bar-north-little-rock-2,North Bar,https://s3-media3.fl.yelpcdn.com/bphoto/-i-SL1...,False,https://www.yelp.com/biz/north-bar-north-littl...,186,"[{'alias': 'tradamerican', 'title': 'American ...",4.5,"{'latitude': 34.7906015, 'longitude': -92.2595...",[delivery],$$,"{'address1': '3812 Jfk Blvd', 'address2': None...",+15014201117,(501) 420-1117,10403.242520
4,SunfITwfSe2KqAFHbYJDlQ,wayback-burgers-little-rock,Wayback Burgers,https://s3-media1.fl.yelpcdn.com/bphoto/BWAwRm...,False,https://www.yelp.com/biz/wayback-burgers-littl...,41,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.0,"{'latitude': 34.7594396, 'longitude': -92.453074}","[pickup, delivery]",$$,"{'address1': '16900 Chenal Pkwy', 'address2': ...",+15014045858,(501) 404-5858,11100.640460


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
415,15q3XqdDRIR_WRB4VZC4dw,waffle-house-jacksonville-10,Waffle House,https://s3-media3.fl.yelpcdn.com/bphoto/Wy4YEt...,False,https://www.yelp.com/biz/waffle-house-jacksonv...,15,"[{'alias': 'diners', 'title': 'Diners'}, {'ali...",2.5,"{'latitude': 34.8687896, 'longitude': -92.128658}",[delivery],NaN,"{'address1': '116 Marshall Rd', 'address2': No...",+15019827986,(501) 982-7986,25086.276523
416,9YclROpOLTUbsKKjq3N-0Q,mcdonalds-sherwood-3,McDonald's,https://s3-media3.fl.yelpcdn.com/bphoto/D4cDPX...,False,https://www.yelp.com/biz/mcdonalds-sherwood-3?...,6,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",1.0,"{'latitude': 34.8306192290247, 'longitude': -9...",[delivery],$,"{'address1': '8400 Warden Rd', 'address2': '',...",+15018356323,(501) 835-6323,18596.254736
417,obmg-MELIZCa47VrVFz0xg,jordans-country-bar-b-q-bryant,Jordan's Country Bar-B Q,https://s3-media3.fl.yelpcdn.com/bphoto/vPVPKs...,False,https://www.yelp.com/biz/jordans-country-bar-b...,39,"[{'alias': 'bbq', 'title': 'Barbeque'}]",2.5,"{'latitude': 34.59564, 'longitude': -92.4887}",[delivery],$,"{'address1': '110 N Reynolds Rd', 'address2': ...",+15018476167,(501) 847-6167,19755.183344
418,d14AiJMSS7NndCre9jTT2Q,waffle-house-greensboro-2,Waffle House,https://s3-media3.fl.yelpcdn.com/bphoto/NRXK-X...,False,https://www.yelp.com/biz/waffle-house-greensbo...,35,"[{'alias': 'breakfast_brunch', 'title': 'Break...",3.5,"{'latitude': 34.872062, 'longitude': -92.109731}",[],$,"{'address1': '4302 Big Tree Way', 'address2': ...",+13362940035,(336) 294-0035,26654.981167
419,6xQN0yXxxkvlNSQPRjAI_w,mcdonalds-cabot-4,McDonald's,https://s3-media4.fl.yelpcdn.com/bphoto/qFvYpk...,False,https://www.yelp.com/biz/mcdonalds-cabot-4?adj...,20,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",1.5,"{'latitude': 34.9757433546351, 'longitude': -9...",[],$,"{'address1': '1001 W Main St', 'address2': '',...",+15018435559,(501) 843-5559,40021.423628


In [21]:
# check for duplicate results
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             420 non-null    object 
 1   alias          420 non-null    object 
 2   name           420 non-null    object 
 3   image_url      420 non-null    object 
 4   is_closed      420 non-null    bool   
 5   url            420 non-null    object 
 6   review_count   420 non-null    int64  
 7   categories     420 non-null    object 
 8   rating         420 non-null    float64
 9   coordinates    420 non-null    object 
 10  transactions   420 non-null    object 
 11  price          317 non-null    object 
 12  location       420 non-null    object 
 13  phone          420 non-null    object 
 14  display_phone  420 non-null    object 
 15  distance       420 non-null    float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 49.8+ KB


In [22]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [23]:
final_df.to_csv(f'Data/final_results_{LOCATION}_{TERM}.csv.gz', compression='gzip',index=False)